In [ ]:
# --- 0. Install Libraries ---
import subprocess
import sys

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

packages_to_check = ['transformers', 'openpyxl', 'tqdm']
for package in packages_to_check:
    install_if_missing(package)

# --- 1. Import Libraries ---
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_text
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
import os
warnings.filterwarnings('ignore')

# --- 2. Kaggle Environment Detection ---
def setup_kaggle_environment():
    is_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None
    if is_kaggle:
        print("Kaggle environment detected!")
        input_dir = '/kaggle/input'
        working_dir = '/kaggle/working'
        dataset_files = []
        if os.path.exists(input_dir):
            for root, dirs, files in os.walk(input_dir):
                for file in files:
                    if file.endswith(('.xlsx', '.csv')):
                        dataset_files.append(os.path.join(root, file))
        print(f"Available dataset files: {dataset_files}")
        file_path = dataset_files[0] if dataset_files else '/kaggle/input/csic_database.xlsx'
        return {'file_path': file_path, 'output_dir': working_dir, 'is_kaggle': True}
    else:
        print("Local environment detected!")
        return {'file_path': './csic_database.xlsx', 'output_dir': './', 'is_kaggle': False}

env_config = setup_kaggle_environment()

CONFIG = {
    'file_path': env_config['file_path'],
    'output_dir': env_config['output_dir'],
    'batch_size': 16 if torch.cuda.is_available() else 8,
    'max_length': 512,
    'learning_rate': 2e-5,
    'num_epochs': 3,
    'max_depth': 10,
    'test_split': 0.2,
    'random_seed': 42,
    'is_kaggle': env_config['is_kaggle']
}

torch.manual_seed(CONFIG['random_seed'])
np.random.seed(CONFIG['random_seed'])
print(f"Configuration: {CONFIG}")

# --- 4. Data Loading ---
def load_and_preprocess_data(file_path):
    try:
        print(f"Loading dataset from: {file_path}")
        if not os.path.exists(file_path) and CONFIG['is_kaggle']:
            input_dir = '/kaggle/input'
            print(f"File not found. Searching in {input_dir}...")
            for root, dirs, files in os.walk(input_dir):
                for file in files:
                    if 'csic' in file.lower() or 'dataset' in file.lower():
                        file_path = os.path.join(root, file)
                        break
        if file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            try: df = pd.read_excel(file_path)
            except: df = pd.read_csv(file_path)
        print(f"Dataset loaded! Shape: {df.shape}")
        label_columns = ['classification','label','class','target']
        label_col = next((c for c in label_columns if c in df.columns), None)
        if label_col:
            print(f"Label distribution:\n{df[label_col].value_counts()}")
        return df
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

# --- 5. Preprocess ---
def preprocess_data(df):
    print("Preprocessing data...")
    label_columns = ['classification','label','class','target']
    label_col = next((c for c in label_columns if c in df.columns), None)
    if label_col and label_col != 'label':
        df.rename(columns={label_col:'label'}, inplace=True)
    length_columns = ['lenght','length','len']
    for col in length_columns: df.drop(col, axis=1, inplace=True, errors='ignore')
    potential_text_cols = [
        'Method','method','HTTP_Method','User-Agent','user-agent','useragent','User_Agent',
        'Pragma','pragma','Cache-Control','cache-control','Cache_Control','Accept','accept',
        'Accept-encoding','accept-encoding','Accept_Encoding','Accept-charset','accept-charset','Accept_Charset',
        'language','Language','lang','host','Host','hostname','cookie','Cookie','cookies',
        'content-type','Content-Type','Content_Type','contenttype','connection','Connection','URL','url','uri','path',
        'content','Content','payload','data','body']
    available_text_cols = []
    for col in df.columns:
        if col in potential_text_cols or any(k in col.lower() for k in ['method','agent','url','content','header']):
            available_text_cols.append(col)
            df[col] = df[col].astype(str).fillna('')
    print(f"Available text columns: {available_text_cols}")
    if available_text_cols:
        combined_parts = [f'{col}: '+df[col].astype(str) for col in available_text_cols]
        df['combined_text'] = combined_parts[0]
        for part in combined_parts[1:]:
            df['combined_text'] += ' '+part
    else:
        text_cols = df.select_dtypes(include=['object']).columns.tolist()
        if 'label' in text_cols: text_cols.remove('label')
        if text_cols:
            print(f"Using all object columns as text: {text_cols}")
            combined_parts = [f'{col}: '+df[col].astype(str).fillna('') for col in text_cols]
            df['combined_text'] = combined_parts[0]
            for part in combined_parts[1:]:
                df['combined_text'] += ' '+part
        else:
            print("No text columns found!")
            return None,None,None
    if 'label' not in df.columns:
        print("No 'label' column found!")
        return None,None,None
    combined_text = df['combined_text']
    y_raw = df['label']
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_raw)
    print(f"Classes: {label_encoder.classes_}")
    return combined_text,y,label_encoder

# --- 6. Dataset ---
class CSICBertDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings; self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        item = {k:v[idx] for k,v in self.encodings.items()}
        item['labels'] = self.labels[idx]; return item

# --- 7. Model ---
class BertClassifier(nn.Module):
    def __init__(self, n_classes, dropout_rate=0.3):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    def forward(self,input_ids,attention_mask):
        outputs = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        cls_embedding = self.dropout(outputs.last_hidden_state[:,0,:])
        logits = self.classifier(cls_embedding)
        return logits,cls_embedding

# --- 8. Training with checkpoint ---
def train_model(model, train_loader, val_loader, device, config,
                optimizer, scheduler, resume_epoch=0,
                train_losses=None, val_losses=None, val_accuracies=None):
    if train_losses is None: train_losses=[]
    if val_losses is None: val_losses=[]
    if val_accuracies is None: val_accuracies=[]
    criterion = nn.CrossEntropyLoss()
    for epoch in range(resume_epoch, config['num_epochs']):
        model.train(); total_train_loss=0
        for batch in tqdm(train_loader,desc=f'Epoch {epoch+1}/{config["num_epochs"]} [Train]'):
            input_ids=batch['input_ids'].to(device)
            attention_mask=batch['attention_mask'].to(device)
            labels=batch['labels'].to(device)
            optimizer.zero_grad()
            logits,_=model(input_ids,attention_mask)
            loss=criterion(logits,labels); loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            optimizer.step(); scheduler.step()
            total_train_loss+=loss.item()
        avg_train_loss=total_train_loss/len(train_loader)
        train_losses.append(avg_train_loss)
        # validation
        model.eval(); total_val_loss=0; correct=0; total=0
        with torch.no_grad():
            for batch in tqdm(val_loader,desc=f'Epoch {epoch+1}/{config["num_epochs"]} [Val]'):
                input_ids=batch['input_ids'].to(device)
                attention_mask=batch['attention_mask'].to(device)
                labels=batch['labels'].to(device)
                logits,_=model(input_ids,attention_mask)
                loss=criterion(logits,labels); total_val_loss+=loss.item()
                preds=torch.argmax(logits,dim=1)
                correct+=(preds==labels).sum().item(); total+=labels.size(0)
        avg_val_loss=total_val_loss/len(val_loader); val_acc=correct/total
        val_losses.append(avg_val_loss); val_accuracies.append(val_acc)
        print(f"Epoch {epoch+1}: Train Loss {avg_train_loss:.4f} Val Loss {avg_val_loss:.4f} Val Acc {val_acc:.4f}")
        # save checkpoint
        checkpoint_path=os.path.join(config['output_dir'],'bert_checkpoint.pt')
        torch.save({
            'epoch':epoch+1,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
            'scheduler_state_dict':scheduler.state_dict(),
            'train_losses':train_losses,
            'val_losses':val_losses,
            'val_accuracies':val_accuracies
        },checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")
    return train_losses,val_losses,val_accuracies

# --- 9. Evaluation ---
def evaluate_model(model, test_loader, device, label_encoder, config):
    model.eval(); all_preds=[]; all_trues=[]
    with torch.no_grad():
        for batch in tqdm(test_loader,desc='Evaluating'):
            input_ids=batch['input_ids'].to(device)
            attention_mask=batch['attention_mask'].to(device)
            labels=batch['labels'].to(device)
            logits,_=model(input_ids,attention_mask)
            preds=torch.argmax(logits,dim=1)
            all_preds.extend(preds.cpu().tolist()); all_trues.extend(labels.cpu().tolist())
    accuracy=accuracy_score(all_trues,all_preds)
    f1w=f1_score(all_trues,all_preds,average='weighted')
    print(f"Test Accuracy: {accuracy:.4f} Weighted F1: {f1w:.4f}")
    print(classification_report(all_trues,all_preds,target_names=label_encoder.classes_.astype(str)))
    cm=confusion_matrix(all_trues,all_preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm,annot=True,fmt='d',cmap='Blues',
                xticklabels=label_encoder.classes_.astype(str),
                yticklabels=label_encoder.classes_.astype(str))
    plt.title('Confusion Matrix'); plt.ylabel('True'); plt.xlabel('Predicted')
    plt.tight_layout()
    plt.savefig(os.path.join(config['output_dir'],'confusion_matrix.png'),dpi=300)
    plt.show()
    return all_preds,all_trues,accuracy,f1w

# --- 11. Main ---
def main():
    print("="*60)
    print("CSIC BERT CLASSIFIER WITH RESUME SUPPORT")
    print("="*60)
    df=load_and_preprocess_data(CONFIG['file_path'])
    if df is None: return
    combined_text,y,label_encoder=preprocess_data(df)
    if combined_text is None: return
    y_tensor=torch.tensor(y,dtype=torch.long)
    tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_inputs=tokenizer(
        combined_text.tolist(),
        padding='max_length',truncation=True,max_length=CONFIG['max_length'],return_tensors="pt")
    dataset=CSICBertDataset(tokenized_inputs,y_tensor)
    total_size=len(dataset)
    test_size=int(CONFIG['test_split']*total_size)
    train_val_size=total_size-test_size
    val_size=int(0.1*total_size)
    train_size=train_val_size-val_size
    train_ds,val_ds,test_ds=random_split(dataset,[train_size,val_size,test_size],
                                        generator=torch.Generator().manual_seed(CONFIG['random_seed']))
    print(f"Splits - Train: {train_size} Val: {val_size} Test: {test_size}")
    train_loader=DataLoader(train_ds,batch_size=CONFIG['batch_size'],shuffle=True)
    val_loader=DataLoader(val_ds,batch_size=CONFIG['batch_size'],shuffle=False)
    test_loader=DataLoader(test_ds,batch_size=CONFIG['batch_size'],shuffle=False)
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=BertClassifier(n_classes=len(label_encoder.classes_)).to(device)
    optimizer=torch.optim.AdamW(model.parameters(),lr=CONFIG['learning_rate'])
    total_steps=len(train_loader)*CONFIG['num_epochs']
    scheduler=get_linear_schedule_with_warmup(
        optimizer,num_warmup_steps=int(0.1*total_steps),num_training_steps=total_steps)
    # resume logic
    checkpoint_path=os.path.join(CONFIG['output_dir'],'bert_checkpoint.pt')
    resume_epoch=0; train_losses=val_losses=val_accuracies=None
    if os.path.exists(checkpoint_path):
        print(f"Resuming from checkpoint: {checkpoint_path}")
        checkpoint=torch.load(checkpoint_path,map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        resume_epoch=checkpoint['epoch']
        train_losses=checkpoint['train_losses']; val_losses=checkpoint['val_losses']; val_accuracies=checkpoint['val_accuracies']
    train_losses,val_losses,val_accuracies=train_model(
        model,train_loader,val_loader,device,CONFIG,optimizer,scheduler,resume_epoch,
        train_losses,val_losses,val_accuracies)
    # evaluate
    all_preds,all_trues,accuracy,f1w=evaluate_model(model,test_loader,device,label_encoder,CONFIG)
    # save final model
    model_path=os.path.join(CONFIG['output_dir'],'bert_classifier_model.pt')
    torch.save({'model_state_dict':model.state_dict(),
                'label_encoder':label_encoder,
                'config':CONFIG,
                'test_accuracy':accuracy,
                'f1_score':f1w},model_path)
    print(f"Model saved to {model_path}")
    print("Done.")

if __name__=="__main__":
    main() 

In [ ]:
# ==============================================================================
# Complete BERT → DistilBERT + MLP Knowledge Distillation Pipeline
# CSIC 2010 Web Application Attack Detector (Adaptive WAF)
# ==============================================================================

# --- 0. Setup and Imports (omitted for brevity, assume the user's provided imports) ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report, accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier, export_text # XAI Import
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    BertTokenizer, BertModel,
    DistilBertModel,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from tqdm.auto import tqdm
import warnings
import os

warnings.filterwarnings('ignore')
torch.manual_seed(42)
np.random.seed(42)

# --- 1. Configuration & Environment ---
CHECKPOINT_PATH = '/kaggle/working/bert_classifier_model.pt'
DATASET_PATH = '/kaggle/input/csic-2010-web-application-attacks/csic_database.csv'
BEST_MODEL_PATH = '/kaggle/working/best_student_waf_model.pt'
MAX_LENGTH = 512
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5
OUTPUT_DIR = '/kaggle/working'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
XAI_MAX_DEPTH = 10

# --- 2. Dataset and Data Loading (TextDataset and load_csic_dataset functions remain the same) ---
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
        
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

def load_csic_dataset(file_path):
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: Dataset not found at {file_path}. Please check the path.")
        return None, None, None

    text_columns = ['Method', 'User-Agent', 'Pragma', 'Cache-Control', 'Accept',
                    'Accept-encoding', 'Accept-charset', 'language', 'host',
                    'cookie', 'content-type', 'connection', 'content', 'URL']
    
    df['combined_text'] = ''
    for col in text_columns:
        if col in df.columns:
            df['combined_text'] += df[col].fillna('').astype(str) + ' '
    
    df['combined_text'] = df['combined_text'].str.strip()
    
    texts = df['combined_text'].values
    labels_raw = df['classification'].values
    
    le = LabelEncoder()
    labels = le.fit_transform(labels_raw)

    print(f"Dataset loaded! Shape: {df.shape}")
    print(f"Label distribution:\n{df['classification'].value_counts()}")
    return texts, labels, le

# --- 3. Model Architectures (TeacherBERT, StudentDistilBERT, StudentMLP functions remain the same) ---
class TeacherBERT(nn.Module):
    def __init__(self, n_classes, model_name='bert-base-uncased', dropout_rate=0.3):
        super(TeacherBERT, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = self.dropout(outputs.last_hidden_state[:, 0, :])
        logits = self.classifier(cls_embedding)
        return logits

class StudentDistilBERT(nn.Module):
    def __init__(self, model_name='distilbert-base-uncased', num_classes=2, dropout=0.1):
        super(StudentDistilBERT, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.distilbert.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output_dropped = self.dropout(pooled_output)
        logits = self.classifier(pooled_output_dropped)
        return logits, pooled_output

class StudentMLP(nn.Module):
    def __init__(self, vocab_size=30522, embed_dim=128, hidden_dims=[256, 128], 
                 num_classes=2, dropout=0.3):
        super(StudentMLP, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        
        layers = []
        input_dim = embed_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            input_dim = hidden_dim
            
        layers.append(nn.Linear(input_dim, num_classes))
        self.mlp = nn.Sequential(*layers)
        
    def forward(self, input_ids, attention_mask):
        embeddings = self.embedding(input_ids)
        mask = attention_mask.unsqueeze(-1).float()
        masked_embeddings = embeddings * mask
        pooled = masked_embeddings.sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)
        pooled = self.dropout(pooled)
        logits = self.mlp(pooled)
        return logits

# --- 4. Distillation Loss and Training Function (DistillationLoss and train_student_model functions remain the same) ---
class DistillationLoss(nn.Module):
    def __init__(self, alpha=0.7, temperature=4.0):
        super(DistillationLoss, self).__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.kl_div = nn.KLDivLoss(reduction='batchmean')
        self.ce_loss = nn.CrossEntropyLoss()
        
    def forward(self, student_logits, teacher_logits, labels):
        teacher_probs = F.softmax(teacher_logits / self.temperature, dim=1)
        student_log_probs = F.log_softmax(student_logits / self.temperature, dim=1)
        distillation_loss = self.kl_div(student_log_probs, teacher_probs) * (self.temperature ** 2)
        student_loss = self.ce_loss(student_logits, labels)
        total_loss = self.alpha * distillation_loss + (1 - self.alpha) * student_loss
        return total_loss, distillation_loss, student_loss

def train_student_model(student_model, teacher_model, train_loader, val_loader, device, name):
    student_model.to(device)
    teacher_model.to(device)
    teacher_model.eval()
    
    optimizer = AdamW(student_model.parameters(), lr=LEARNING_RATE)
    distillation_criterion = DistillationLoss(alpha=0.7, temperature=4.0)
    
    total_steps = len(train_loader) * NUM_EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )
    
    train_losses = []
    val_accuracies = []
    
    print(f"\n--- Starting Distillation for {name} (Epochs: {NUM_EPOCHS}) ---")
    for epoch in range(NUM_EPOCHS):
        student_model.train()
        total_loss = 0
        
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} [{name}]')
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids, attention_mask)
                
            if name == 'DistilBERT':
                student_logits, _ = student_model(input_ids, attention_mask)
            else:
                student_logits = student_model(input_ids, attention_mask)
            
            loss, dist_loss, student_loss = distillation_criterion(
                student_logits, teacher_logits, labels
            )
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student_model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
            total_loss += loss.item()
            
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Dist': f'{dist_loss.item():.4f}',
            })
            
        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        val_accuracy, _ = evaluate_model(student_model, val_loader, device, is_distilbert=(name=='DistilBERT'))
        val_accuracies.append(val_accuracy)
        
        print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
        
    return train_losses, val_accuracies

# --- 5. Evaluation, Checkpoint Loading, and Plotting (load_teacher_checkpoint, evaluate_model, plot_roc_comparison functions remain the same) ---
def load_teacher_checkpoint(checkpoint_path, n_classes):
    print("Loading teacher checkpoint...")
    
    teacher_model = TeacherBERT(n_classes=n_classes)
    teacher_model.to(DEVICE)
    
    try:
        teacher_ckpt = torch.load(checkpoint_path, map_location=DEVICE, weights_only=False)
        
        if 'model_state_dict' in teacher_ckpt:
            state_dict = teacher_ckpt['model_state_dict']
            
            new_state_dict = {}
            for k, v in state_dict.items():
                if k.startswith('module.'):
                    k = k[7:]
                new_state_dict[k] = v
            
            teacher_model.load_state_dict(new_state_dict)
            
            print("✓ Teacher weights loaded successfully!")
            return teacher_model, teacher_ckpt.get('label_encoder'), teacher_ckpt.get('config')
        else:
            print("✗ Checkpoint file is missing 'model_state_dict'. Cannot load weights.")
            return None, None, None
            
    except Exception as e:
        print(f"✗ Failed to load teacher checkpoint: {e}")
        print("Using randomly initialized BERT Teacher. Expect poor performance.")
        return teacher_model, None, None

def evaluate_model(model, dataloader, device, return_probs=False, is_distilbert=False):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    total_loss = 0
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            if is_distilbert and isinstance(model, StudentDistilBERT):
                logits, _ = model(input_ids, attention_mask)
            else:
                logits = model(input_ids, attention_mask)
            
            loss = criterion(logits, labels)
            total_loss += loss.item()
            
            probs = F.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
    accuracy = accuracy_score(all_labels, all_preds)
    avg_loss = total_loss / len(dataloader)
    
    if return_probs:
        return accuracy, avg_loss, all_labels, all_preds, all_probs
    return accuracy, avg_loss

def plot_roc_comparison(models_data, save_path=None):
    plt.figure(figsize=(12, 8))
    colors = ['blue', 'red', 'green', 'orange']
    
    for i, (name, labels, probs) in enumerate(models_data):
        y_score = np.array(probs)[:, 1] if len(probs[0]) > 1 else np.array(probs)
        
        fpr, tpr, _ = roc_curve(labels, y_score)
        roc_auc = auc(fpr, tpr)
        
        plt.plot(fpr, tpr, color=colors[i % len(colors)], lw=2,
                 label=f'{name} (AUC = {roc_auc:.4f})')
        
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curve Comparison: Teacher vs Student Models', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=11)
    plt.grid(True, alpha=0.3)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

# --- 6. Main Distillation Pipeline ---

def main_distillation_pipeline():
    print("=" * 80)
    print("BERT → DistilBERT + MLP Knowledge Distillation Pipeline")
    print("CSIC 2010 Web Application Attacks Dataset")
    print("=" * 80)
    
    # Load and preprocess data
    texts, encoded_labels, label_encoder = load_csic_dataset(DATASET_PATH)
    if texts is None: return

    num_classes = len(label_encoder.classes_)
    
    # --- CRITICAL CHANGE: Use ALL samples, remove subsetting logic ---
    # The splitting will now use the entire dataset (approx. 61k samples)
    # Split: 60% Train, 20% Val, 20% Test
    
    X_train, X_test, y_train, y_test = train_test_split(
        texts, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.25, random_state=42, stratify=y_train 
    )
    
    print(f"Data Splits - Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
    
    # Initialize tokenizers and DataLoaders
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    train_dataset = TextDataset(X_train, y_train, tokenizer, MAX_LENGTH)
    val_loader = DataLoader(TextDataset(X_val, y_val, tokenizer, MAX_LENGTH), batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(TextDataset(X_test, y_test, tokenizer, MAX_LENGTH), batch_size=BATCH_SIZE, shuffle=False)
    
    # --- Step 1: Initialize and Load Teacher Model ---
    teacher_model, _, _ = load_teacher_checkpoint(CHECKPOINT_PATH, num_classes)
    if teacher_model is None:
        print("Cannot proceed with distillation without a valid Teacher model.")
        return

    # --- Step 2: Initialize Student Models ---
    student_distilbert = StudentDistilBERT(num_classes=num_classes)
    student_mlp = StudentMLP(num_classes=num_classes, vocab_size=tokenizer.vocab_size)
    
    print(f"\nModel Parameters (Teacher: {sum(p.numel() for p in teacher_model.parameters()):,} | DistilBERT: {sum(p.numel() for p in student_distilbert.parameters()):,} | MLP: {sum(p.numel() for p in student_mlp.parameters()):,})")

    # --- Step 3: Train Students ---
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    distilbert_metrics = train_student_model(student_distilbert, teacher_model, train_loader, val_loader, DEVICE, 'DistilBERT')
    mlp_metrics = train_student_model(student_mlp, teacher_model, train_loader, val_loader, DEVICE, 'MLP')
    
    # --- Step 4: Final Evaluation and Model Saving ---
    print("\n" + "="*80)
    print("FINAL EVALUATION on Test Set & BEST MODEL SAVING")
    print("="*80)
    
    models = {
        'Teacher BERT (Corrected)': teacher_model,
        'Student DistilBERT': student_distilbert,
        'Student MLP': student_mlp
    }
    
    best_f1 = -1
    best_model_name = ""
    best_model_data = None
    models_roc_data = []

    for name, model in models.items():
        is_distilbert_eval = ("DistilBERT" in name)
        
        accuracy, loss, labels, preds, probs = evaluate_model(
            model, test_loader, DEVICE, return_probs=True, is_distilbert=is_distilbert_eval
        )
        
        models_roc_data.append((name, labels, probs))
        f1w = f1_score(labels, preds, average='weighted')
        
        print(f"\n{name} - Accuracy: {accuracy:.4f}, Weighted F1: {f1w:.4f}, Loss: {loss:.4f}")
        print(classification_report(labels, preds, target_names=label_encoder.classes_.astype(str)))

        if "Teacher" not in name and f1w > best_f1:
            best_f1 = f1w
            best_model_name = name
            best_model_data = {
                'model_state_dict': model.state_dict(),
                'label_encoder': label_encoder,
                'config': {'max_length': MAX_LENGTH, 'batch_size': BATCH_SIZE},
                'test_accuracy': accuracy,
                'f1_score': f1w,
                'model_architecture': name
            }

    if best_model_data:
        torch.save(best_model_data, BEST_MODEL_PATH)
        print("\n" + "="*80)
        print(f"✅ FINAL DEPLOYMENT MODEL SAVED: {best_model_name} (F1: {best_f1:.4f})")
        print(f"File: {BEST_MODEL_PATH}")
        print("="*80)
    else:
        print("\n❌ Could not save the best model.")


    # --- Step 5: Visualization ---
    print("\n--- Visualizing ROC Curves ---")
    plot_roc_comparison(models_roc_data, os.path.join(OUTPUT_DIR, 'roc_comparison.png'))
    
    print("\n✓ Cybersecurity knowledge distillation pipeline completed successfully!")
    
    # --- Step 6: Trigger XAI Agent ---
    if best_model_data and 'DistilBERT' in best_model_data.get('model_architecture', ''):
        main_xai_agent(X_test, y_test, label_encoder.classes_.tolist())
    else:
        print("\nSkipping XAI Agent: The best model was not DistilBERT or data was unavailable.")

# --- 7. XAI Core Functions (Extracted from XAI Agent - functions remain the same) ---

def extract_features(model, dataloader, device) -> np.ndarray:
    model.eval()
    all_features = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting Features for XAI"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            _, features = model(input_ids, attention_mask)
            all_features.append(features.cpu().numpy())
            
    return np.concatenate(all_features, axis=0)

def generate_xai_rules(X_features: np.ndarray, y_labels: np.ndarray, feature_names: list, class_names: list) -> str:
    print("\nTraining Decision Tree Surrogate Model...")
    
    dt_model = DecisionTreeClassifier(max_depth=XAI_MAX_DEPTH, random_state=42)
    dt_model.fit(X_features, y_labels)
    
    dt_preds = dt_model.predict(X_features)
    dt_acc = accuracy_score(y_labels, dt_preds)
    print(f"Decision Tree (Surrogate) Accuracy on Extracted Features: {dt_acc:.4f}")
    
    rules = export_text(
        dt_model, 
        feature_names=feature_names, 
        class_names=class_names
    )
    return rules

def main_xai_agent(X_test, y_test, class_names_list):
    print("\n" + "="*80)
    print("XAI AGENT: Rule Generation for Adaptive WAF (Surrogate Model)")
    print("="*80)
    
    checkpoint = torch.load(BEST_MODEL_PATH, map_location=DEVICE)
    num_classes = len(class_names_list)
    
    model = StudentDistilBERT(num_classes=num_classes).to(DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Loading '{checkpoint.get('model_architecture')}' with F1-Score: {checkpoint['f1_score']:.4f} for XAI...")

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    test_dataset = TextDataset(X_test, y_test, tokenizer, MAX_LENGTH)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    X_features = extract_features(model, test_loader, DEVICE)
    feature_names = [f'CLS_Dim_{i}' for i in range(X_features.shape[1])]
    
    print(f"Features extracted! Shape: {X_features.shape}")

    xai_rules = generate_xai_rules(X_features, y_test, feature_names, class_names_list)
    
    rules_path = os.path.join(OUTPUT_DIR, 'waf_xai_rules.txt')
    with open(rules_path, 'w') as f:
        f.write(xai_rules)
        
    print("\n" + "="*80)
    print("✅ XAI RULE GENERATION COMPLETE")
    print(f"Rules saved to: {rules_path}")
    print("Sample Rules (Decision Tree Surrogate):")
    print("="*80)
    print('\n'.join(xai_rules.split('\n')[:15]))
    print("... (Rules Truncated) ...")
    print("="*80)

if __name__ == "__main__":
    main_distillation_pipeline()